In [1]:
main_repo_dir = os.path.abspath(os.path.join('../..'))
sys.path.append(os.path.join(main_repo_dir, 'src'))

In [2]:
import datetime

In [3]:
import pandas

In [4]:
import utils

In [314]:
with open(os.path.join(main_repo_dir, "main_run.log"), "r") as f:
    log = f.readlines()

In [315]:
dates, entries = zip(*[(' '.join(line.split()[:3]), ' '.join(line.split()[3:])) for line in log])

In [316]:
dates = [datetime.datetime.strptime(date, '%m/%d/%Y %I:%M:%S %p') for date in dates]

In [317]:
log = pandas.DataFrame(data = {'entry' : entries}, index=dates)

In [318]:
log['day'] = [ent.strftime('%Y-%m-%d') for ent in log.index]

In [319]:
# Define which entry we're in
# There's got to be a better way to do this with an iterator...
entry_ind = -1

def update_entry(x):
    global entry_ind
    if x['entry'] == "Retrieving contents...":
        entry_ind += 1
    return(entry_ind)


# Flag Errors
def flag_error(x):
    text = x.entry.lower()
    if text.find('warning') > -1 or text.find('error') > -1:
        return(True)
    else:
        return(False)
    

def extract_err_class(error_text):
    start = error_text.find('<class ')
    end = error_text.find("'>:")
    if start > -1 and end > -1:
        err_class = error_text[start : (end + 2)]
    else:
        err_class = ''
    return(err_class)

In [320]:
log['entry_ind'] = log.apply(lambda x: update_entry(x), axis=1)
log['is_error'] = log.apply(lambda x: flag_error(x), axis=1)
log['err_class'] = log.entry.apply(lambda x: extract_err_class(x))

In [321]:
log.shape

(179534, 5)

In [322]:
log.head()

,entry,day,entry_ind,is_error,err_class
2017-01-20 12:49:21,Retrieving contents...,2017-01-20,0,False,
2017-01-20 12:49:21,Total feeedss to visit: 65,2017-01-20,0,False,
2017-01-20 12:49:22,IP Used: b'87.118.122.30',2017-01-20,0,False,
2017-01-20 12:49:26,Total entries retrieved from http://phys.org/r...,2017-01-20,0,False,
2017-01-20 12:49:26,Total new links added: 30,2017-01-20,0,False,


In [323]:
log.tail()

,entry,day,entry_ind,is_error,err_class
2017-03-14 06:09:06,Feed http://www.ft.com/rss/management/connecte...,2017-03-14,491,True,<class 'urllib.error.HTTPError'>
2017-03-14 06:09:08,No contents found,2017-03-14,491,False,
2017-03-14 06:09:10,Total entries retrieved from http://www.ft.com...,2017-03-14,491,False,
2017-03-14 06:09:10,Total new links added: 0,2017-03-14,491,False,
2017-03-14 06:09:10,Total new links added from all feeds: 86,2017-03-14,491,False,


## Investiate Errors

In [324]:
errors = log[log.is_error==True].copy()

In [325]:
errors.shape

(5653, 5)

In [326]:
errors.head()

,entry,day,entry_ind,is_error,err_class
2017-01-30 08:03:45,Error encountered: <class 'mysql.connector.err...,2017-01-30,82,True,<class 'mysql.connector.errors.DataError'>
2017-01-30 14:03:24,Error encountered: <class 'mysql.connector.err...,2017-01-30,88,True,<class 'mysql.connector.errors.DataError'>
2017-01-30 17:02:53,Error encountered: <class 'mysql.connector.err...,2017-01-30,91,True,<class 'mysql.connector.errors.DataError'>
2017-01-31 06:05:57,Error encountered: <class 'mysql.connector.err...,2017-01-31,93,True,<class 'mysql.connector.errors.DataError'>
2017-01-31 09:03:14,Error encountered: <class 'mysql.connector.err...,2017-01-31,96,True,<class 'mysql.connector.errors.DataError'>


In [327]:
errors.tail()

,entry,day,entry_ind,is_error,err_class
2017-03-13 18:08:50,Feed http://www.ft.com/rss/management/connecte...,2017-03-13,489,True,<class 'urllib.error.HTTPError'>
2017-03-14 05:08:13,Feed http://www.ft.com/rss/world/mideast/econo...,2017-03-14,490,True,<class 'urllib.error.HTTPError'>
2017-03-14 05:09:12,Feed http://www.ft.com/rss/management/connecte...,2017-03-14,490,True,<class 'urllib.error.HTTPError'>
2017-03-14 06:08:19,Feed http://www.ft.com/rss/world/mideast/econo...,2017-03-14,491,True,<class 'urllib.error.HTTPError'>
2017-03-14 06:09:06,Feed http://www.ft.com/rss/management/connecte...,2017-03-14,491,True,<class 'urllib.error.HTTPError'>


In [328]:
for e in errors.err_class.unique():
    print(e)

<class 'mysql.connector.errors.DataError'>
<class 'urllib.error.URLError'>
<class 'UnboundLocalError'>
<class 'KeyError'>
<class 'urllib.error.HTTPError'>
<class 'TypeError'>
<class 'mysql.connector.errors.DatabaseError'>


### Key Errors

In [47]:
ke = errors[errors.err_class=="<class 'KeyError'>"].copy()
ke.shape

(1088, 5)

In [48]:
ke.head()

,entry,day,entry_ind,is_error,err_class
2017-02-06 05:04:14,Feed http://hosted.ap.org/lineups/TOPHEADS.rss...,2017-02-06,150,True,<class 'KeyError'>
2017-02-06 05:04:15,Feed http://hosted.ap.org/lineups/USHEADS.rss?...,2017-02-06,150,True,<class 'KeyError'>
2017-02-06 05:04:17,Feed http://hosted.ap.org/lineups/WORLDHEADS.r...,2017-02-06,150,True,<class 'KeyError'>
2017-02-06 05:04:18,Feed http://hosted.ap.org/lineups/POLITICSHEAD...,2017-02-06,150,True,<class 'KeyError'>
2017-02-06 05:04:19,Feed http://hosted.ap.org/lineups/WASHINGTONHE...,2017-02-06,150,True,<class 'KeyError'>


In [49]:
len(ke.entry.unique())

157

In [50]:
ke.tail()

,entry,day,entry_ind,is_error,err_class
2017-02-10 21:02:54,Feed https://phys.org/rss-feed/earth-news/: <c...,2017-02-10,215,True,<class 'KeyError'>
2017-02-10 21:02:56,Feed https://phys.org/rss-feed/earth-news/eart...,2017-02-10,215,True,<class 'KeyError'>
2017-02-10 21:02:58,Feed https://phys.org/rss-feed/earth-news/envi...,2017-02-10,215,True,<class 'KeyError'>
2017-02-10 21:03:00,Feed https://phys.org/rss-feed/science-news/: ...,2017-02-10,215,True,<class 'KeyError'>
2017-02-10 21:03:02,Feed https://phys.org/rss-feed/science-news/ar...,2017-02-10,215,True,<class 'KeyError'>


Seem to be fixed

### Time Type Errors

In [51]:
te = errors[errors.err_class == "<class 'TypeError'>"].copy()
te.shape

(51, 5)

In [52]:
te.tail()

,entry,day,entry_ind,is_error,err_class
2017-03-09 05:05:37,Feed http://feeds.feedburner.com/breitbart?for...,2017-03-09,450,True,<class 'TypeError'>
2017-03-09 06:07:05,Feed http://feeds.feedburner.com/breitbart?for...,2017-03-09,451,True,<class 'TypeError'>
2017-03-09 07:05:53,Feed http://feeds.feedburner.com/breitbart?for...,2017-03-09,452,True,<class 'TypeError'>
2017-03-09 08:06:31,Feed http://feeds.feedburner.com/breitbart?for...,2017-03-09,453,True,<class 'TypeError'>
2017-03-09 09:05:33,Feed http://feeds.feedburner.com/breitbart?for...,2017-03-09,454,True,<class 'TypeError'>


In [53]:
len(te.entry.unique())

1

In [54]:
te.entry[0]

"Feed http://feeds.feedburner.com/breitbart?format=xml: <class 'TypeError'>: ('Tuple or struct_time argument required',)"

In [55]:
sys.path.append(os.path.join(os.path.abspath('../..'), 'src'))
sys.path.append(os.path.join(os.path.abspath('../..'), 'data'))

In [81]:
import utils
import scrape_feeds
import mysql_utils

In [82]:
from importlib import reload

In [133]:
reload(scrape_feeds)

<module 'scrape_feeds' from '/home/immersinn/gits/rssfeed_link_collector/src/scrape_feeds.py'>

In [108]:
feed_data = utils.load_feedlist_data('breitbart_feedlist.xml')

In [109]:
feed_data[0]

{'Link': 'http://feeds.feedburner.com/breitbart?format=xml',
 'RssName': 'All News',
 'Source': 'breitbart',
 'keymap': {'id': 'id',
  'link': 'id',
  'published_parsed': 'published_parsed',
  'summary': 'summary',
  'title': 'title'}}

In [110]:
rss_entry = feed_data[0]

In [111]:
contents = scrape_feeds.get_feed_contents(rss_entry)

In [112]:
len(contents)

46

In [113]:
contents[0]

{'link': 'http://www.breitbart.com/big-government/2017/03/13/state-of-tennessee-files-constitutional-challenge-to-refugee-resettlement-program/',
 'published': datetime.datetime(2017, 3, 13, 12, 24, 36),
 'rss_link': 'http://feeds.feedburner.com/breitbart?format=xml',
 'story_id': 'http://www.breitbart.com/big-government/2017/03/13/state-of-tennessee-files-constitutional-challenge-to-refugee-resettlement-program/',
 'summary': '<a href="http://www.breitbart.com/big-government/2017/03/13/state-of-tennessee-files-constitutional-challenge-to-refugee-resettlement-program/" title="State of Tennessee Files Constitutional Challenge to Refugee Resettlement Program"><img width="200" height="150" src="http://media.breitbart.com/media/2017/03/refugees-america-200x150.jpg" alt="" /></a><br /> The Thomas More Law Center filed a federal lawsuit on behalf of the Tennessee General Assembly and the State of Tennessee in the U.S. District Court for the Western District of Tennessee on Monday challenging

In [115]:
mysql_utils.saveNewLinks(contents)

[]

In [77]:
type(contents[0]['published'])

datetime.datetime

In [78]:
from time import mktime, sleep, gmtime

In [79]:
gmtime()

time.struct_time(tm_year=2017, tm_mon=3, tm_mday=13, tm_hour=20, tm_min=34, tm_sec=15, tm_wday=0, tm_yday=72, tm_isdst=0)

In [80]:
type(gmtime())

time.struct_time

    Currently don't have an active example of the error...seems like every once in awhile an article has a time that is not like a time?  Code now forces all timestamps in "published" attribute to be [datetime.datetime, time.struct_time].

### 'UnboundLocalError'

In [117]:
ule = errors[errors.err_class == "<class 'UnboundLocalError'>"].copy()
ule.shape

(507, 5)

In [118]:
ule.head()

,entry,day,entry_ind,is_error,err_class
2017-02-06 05:03:19,Feed http://www.bbc.co.uk/blogs/theeditors/rss...,2017-02-06,150,True,<class 'UnboundLocalError'>
2017-02-06 06:03:13,Feed http://www.bbc.co.uk/blogs/theeditors/rss...,2017-02-06,151,True,<class 'UnboundLocalError'>
2017-02-06 07:03:09,Feed http://www.bbc.co.uk/blogs/theeditors/rss...,2017-02-06,152,True,<class 'UnboundLocalError'>
2017-02-06 08:04:13,Feed http://www.bbc.co.uk/blogs/theeditors/rss...,2017-02-06,153,True,<class 'UnboundLocalError'>
2017-02-06 09:03:35,Feed http://www.bbc.co.uk/blogs/theeditors/rss...,2017-02-06,154,True,<class 'UnboundLocalError'>


In [119]:
ule.tail()

,entry,day,entry_ind,is_error,err_class
2017-03-13 12:06:09,Feed http://www.bbc.co.uk/blogs/theeditors/rss...,2017-03-13,483,True,<class 'UnboundLocalError'>
2017-03-13 12:08:09,Feed http://rss.cnn.com/rss/cnn_showbiz.rss: <...,2017-03-13,483,True,<class 'UnboundLocalError'>
2017-03-13 13:00:28,Feed http://feeds.foxnews.com/foxnews/latest: ...,2017-03-13,484,True,<class 'UnboundLocalError'>
2017-03-13 13:04:47,Feed http://www.bbc.co.uk/blogs/theeditors/rss...,2017-03-13,484,True,<class 'UnboundLocalError'>
2017-03-13 13:06:14,Feed http://rss.cnn.com/rss/cnn_showbiz.rss: <...,2017-03-13,484,True,<class 'UnboundLocalError'>


In [123]:
for e in ule.entry.unique():
    print(e)
    print('\n')

Feed http://www.bbc.co.uk/blogs/theeditors/rss.xml: <class 'UnboundLocalError'>: ("local variable 'row_nbrs' referenced before assignment",)


Feed http://feeds.foxnews.com/foxnews/latest: <class 'UnboundLocalError'>: ("local variable 'row_nbrs' referenced before assignment",)


Feed https://www.wired.com/feed/: <class 'UnboundLocalError'>: ("local variable 'row_nbrs' referenced before assignment",)


Feed https://www.wired.com/category/gear/feed/: <class 'UnboundLocalError'>: ("local variable 'row_nbrs' referenced before assignment",)


Feed http://feeds.feedburner.com/breitbart?format=xml: <class 'UnboundLocalError'>: ("local variable 'row_nbrs' referenced before assignment",)


Feed http://rss.cnn.com/rss/cnn_showbiz.rss: <class 'UnboundLocalError'>: ("local variable 'row_nbrs' referenced before assignment",)




In [134]:
feed_data = utils.load_feedlist_data('foxnews_feedlist.xml')

In [135]:
rss_entry = feed_data[0]
rss_entry

{'Link': 'http://feeds.foxnews.com/foxnews/latest',
 'Name': 'Latest Headlines',
 'RssName': 'latest',
 'Source': 'foxnews',
 'keymap': {'id': 'link',
  'link': 'link',
  'published_parsed': 'published_parsed',
  'summary': 'summary',
  'title': 'title'}}

In [136]:
contents = scrape_feeds.get_feed_contents(rss_entry)

In [137]:
contents[0]

{'link': 'http://www.foxnews.com/politics/2017/03/13/trump-steps-up-obamacare-repeal-push-meets-with-laws-victims.html',
 'published': datetime.datetime(2017, 3, 13, 20, 41, 19),
 'rss_link': 'http://feeds.foxnews.com/foxnews/latest',
 'story_id': 'http://www.foxnews.com/politics/2017/03/13/trump-steps-up-obamacare-repeal-push-meets-with-laws-victims.html',
 'summary': '',
 'title': 'THE HARD SELL  Trump steps up ObamaCare repeal push, meets with law&rsquo;s &lsquo;victims&rsquo;'}

In [138]:
mysql_utils.saveNewLinks(contents)

[]

In [139]:
for c in contents:
    if len(c['title']) > 200:
        print(len(c['title']))

211
221


    Looks to be an issue with an overflow on the "title" variable...updated the MySQL table to accept up to 300 chars now.  Also added some "clean-up" code that truncates longer titles..
    
    From below: also looks to be issue with other missing data due to other errors...

### MySQL Connector Errors

In [174]:
err01 = "<class 'mysql.connector.errors.DataError'>"
err02 = "<class 'mysql.connector.errors.DatabaseError'>"

#### DB Error

In [175]:
dee = errors[errors.err_class==err02]
dee.shape

(58, 5)

In [143]:
dee.tail()

,entry,day,entry_ind,is_error,err_class
2017-03-02 16:04:57,Error encountered: <class 'mysql.connector.err...,2017-03-02,397,True,<class 'mysql.connector.errors.DatabaseError'>
2017-03-02 17:05:12,Error encountered: <class 'mysql.connector.err...,2017-03-02,398,True,<class 'mysql.connector.errors.DatabaseError'>
2017-03-03 05:04:44,Error encountered: <class 'mysql.connector.err...,2017-03-03,399,True,<class 'mysql.connector.errors.DatabaseError'>
2017-03-03 06:05:44,Error encountered: <class 'mysql.connector.err...,2017-03-03,400,True,<class 'mysql.connector.errors.DatabaseError'>
2017-03-03 07:07:07,Error encountered: <class 'mysql.connector.err...,2017-03-03,401,True,<class 'mysql.connector.errors.DatabaseError'>


In [144]:
len(dee.entry.unique())

1

In [155]:
dee.entry[-1]

'Error encountered: <class \'mysql.connector.errors.DatabaseError\'>: (1366, "1366 (HY000): Incorrect string value: \'\\\\xF0\\\\x9F\\\\x91\\\\x80 T...\' for column \'summary\' at row 1", \'HY000\')'

In [160]:
for e in log['2017-03-03 07:07:00':'2017-03-03 07:07:12']['entry']:
    print(e)

Total entries retrieved from https://www.wired.com/category/underwire/feed/: 10
Total new links added: 0
Total entries retrieved from https://www.wired.com/category/gear/feed/: 10
Error encountered: <class 'mysql.connector.errors.DatabaseError'>: (1366, "1366 (HY000): Incorrect string value: '\\xF0\\x9F\\x91\\x80 T...' for column 'summary' at row 1", 'HY000')
Feed https://www.wired.com/category/gear/feed/: <class 'UnboundLocalError'>: ("local variable 'row_nbrs' referenced before assignment",)
Total entries retrieved from https://www.wired.com/category/reviews/feed/: 10
Total new links added: 0


In [157]:
feed_data = utils.load_feedlist_data('wired_feedlist.xml')

In [161]:
for i,f in enumerate(feed_data):
    print(str(i) + ' ' + f['Link'])

0 https://www.wired.com/feed/
1 https://www.wired.com/category/business/feed/
2 https://www.wired.com/category/design/feed/
3 https://www.wired.com/category/underwire/feed/
4 https://www.wired.com/category/gear/feed/
5 https://www.wired.com/category/reviews/feed/
6 https://www.wired.com/category/science/feed/
7 https://www.wired.com/category/science/science-blogs/feed/
8 https://www.wired.com/category/security/feed/
9 https://www.wired.com/category/transportation/feed/
10 http://feeds.cnevids.com/brand/wired.mrss
11 https://www.wired.com/category/photo/feed/


In [162]:
rss_entry = feed_data[4]
rss_entry

{'Link': 'https://www.wired.com/category/gear/feed/',
 'RssName': 'Tech',
 'Source': 'wired',
 'keymap': {'id': 'id',
  'link': 'link',
  'published_parsed': 'published_parsed',
  'summary': 'summary',
  'title': 'title'}}

In [163]:
contents = scrape_feeds.get_feed_contents(rss_entry)

In [164]:
contents[0]

{'link': 'https://www.wired.com/2017/03/pandora-premium/',
 'published': datetime.datetime(2017, 3, 13, 14, 0, 24),
 'rss_link': 'https://www.wired.com/category/gear/feed/',
 'story_id': 'https://www.wired.com/?p=2171564',
 'summary': '<div class="rss_thumbnail"><img src="https://www.wired.com/wp-content/uploads/2017/03/PandoraHP-660x330.jpg" alt="Pandora Premium Can&#8217;t Hang With Spotify and Apple" /></div>There\'s really nothing groundbreaking about Pandora\'s new on-demand streaming service. The post <a href="https://www.wired.com/2017/03/pandora-premium/">Pandora Premium Can\'t Hang With Spotify and Apple</a> appeared first on <a href="https://www.wired.com">WIRED</a>.',
 'title': 'Pandora Premium Can’t Hang With Spotify and Apple'}

In [167]:
s = '\\xF0\\x9F\\x91\\x80 T...'
print(s)

\xF0\x9F\x91\x80 T...


In [170]:
contents[0]['link'] = 'test'
contents[0]['summary'] = s

In [171]:
contents[0]

{'link': 'test',
 'published': datetime.datetime(2017, 3, 13, 14, 0, 24),
 'rss_link': 'https://www.wired.com/category/gear/feed/',
 'story_id': 'https://www.wired.com/?p=2171564',
 'summary': '\\xF0\\x9F\\x91\\x80 T...',
 'title': 'Pandora Premium Can’t Hang With Spotify and Apple'}

In [172]:
mysql_utils.saveNewLinks(contents)

[64129]

    Seems to be a one-off awkward string error?  The value recorded in the error file must have been transposed from the original value because I cannot get the string to not write to the db...

#### Data Error

In [177]:
dee = errors[errors.err_class==err01]
dee.shape

(491, 5)

In [189]:
dee.tail().index

DatetimeIndex(['2017-03-13 12:06:09', '2017-03-13 12:08:09',
               '2017-03-13 13:00:28', '2017-03-13 13:04:47',
               '2017-03-13 13:06:14'],
              dtype='datetime64[ns]', freq=None)

In [179]:
len(dee.entry.unique())

4

In [180]:
dee.entry.unique()

array([ 'Error encountered: <class \'mysql.connector.errors.DataError\'>: (1406, "1406 (22001): Data too long for column \'story_id\' at row 1", \'22001\')',
       'Error encountered: <class \'mysql.connector.errors.DataError\'>: (1406, "1406 (22001): Data too long for column \'summary\' at row 1", \'22001\')',
       'Error encountered: <class \'mysql.connector.errors.DataError\'>: (1406, "1406 (22001): Data too long for column \'title\' at row 1", \'22001\')',
       'Error encountered: <class \'mysql.connector.errors.DataError\'>: (1406, "1406 (22001): Data too long for column \'link\' at row 1", \'22001\')'], dtype=object)

In [265]:
str(log[str(dee.tail().index[0] - 0*sec)]['entry'][1])

'Feed http://www.bbc.co.uk/blogs/theeditors/rss.xml: <class \'UnboundLocalError\'>: ("local variable \'row_nbrs\' referenced before assignment",)'

In [197]:
sec = pandas.tslib.Timedelta('1 second')

In [229]:
def get_err_contexts(errs):
    contexts = []
    for ind in errs.index:
        i = 1
        success = False
        while not success:
            sub = list(log[str(ind - i*sec)]['entry'])
            if len(sub)== 0:
                i += 1
                if i > 3:
                    success=True
            else:
                success = True
                contexts.append({'index' : str(ind),
                                 'context' :sub})
    return(contexts)

In [230]:
cons = get_err_contexts(dee)

In [252]:
cons[-5:]

[{'context': ['Total entries retrieved from http://www.bbc.co.uk/blogs/theeditors/rss.xml: 15'],
  'index': '2017-03-13 12:06:09'},
 {'context': ['Total entries retrieved from http://rss.cnn.com/rss/cnn_showbiz.rss: 29'],
  'index': '2017-03-13 12:08:09'},
 {'context': ['Total entries retrieved from http://feeds.foxnews.com/foxnews/latest: 38'],
  'index': '2017-03-13 13:00:28'},
 {'context': ['Total entries retrieved from http://feeds.bbci.co.uk/news/have_your_say/rss.xml: 19',
   'Total new links added: 0'],
  'index': '2017-03-13 13:04:47'},
 {'context': ['Total entries retrieved from http://rss.cnn.com/rss/cnn_health.rss: 30',
   'Total new links added: 0'],
  'index': '2017-03-13 13:06:14'}]

In [242]:
for i in range(-5,0):
    print(dee[cons[i]['index']]['entry'][0])

Error encountered: <class 'mysql.connector.errors.DataError'>: (1406, "1406 (22001): Data too long for column 'summary' at row 1", '22001')
Error encountered: <class 'mysql.connector.errors.DataError'>: (1406, "1406 (22001): Data too long for column 'link' at row 1", '22001')
Error encountered: <class 'mysql.connector.errors.DataError'>: (1406, "1406 (22001): Data too long for column 'title' at row 1", '22001')
Error encountered: <class 'mysql.connector.errors.DataError'>: (1406, "1406 (22001): Data too long for column 'summary' at row 1", '22001')
Error encountered: <class 'mysql.connector.errors.DataError'>: (1406, "1406 (22001): Data too long for column 'link' at row 1", '22001')


In [292]:
reload(scrape_feeds)

<module 'scrape_feeds' from '/home/immersinn/gits/rssfeed_link_collector/src/scrape_feeds.py'>

In [293]:
feed_data = utils.load_feedlist_data('bbcnews_feedlist.xml')

In [294]:
for i,f in enumerate(feed_data):
    print(str(i) + ' ' + f['Link'])

0 http://feeds.bbci.co.uk/news/front_page/rss.xml
1 http://feeds.bbci.co.uk/news/world/rss.xml
2 http://feeds.bbci.co.uk/news/uk/rss.xml
3 http://feeds.bbci.co.uk/news/england/rss.xml
4 http://feeds.bbci.co.uk/news/northern_ireland/rss.xml
5 http://feeds.bbci.co.uk/news/scotland/rss.xml
6 http://feeds.bbci.co.uk/news/wales/rss.xml
7 http://feeds.bbci.co.uk/news/business/rss.xml
8 http://feeds.bbci.co.uk/news/health/rss.xml
9 http://feeds.bbci.co.uk/news/education/rss.xml
10 http://feeds.bbci.co.uk/news/science_and_environment/rss.xml
11 http://feeds.bbci.co.uk/news/technology/rss.xml
12 http://feeds.bbci.co.uk/news/magazine/rss.xml
13 http://feeds.bbci.co.uk/news/rss.xml
14 http://feeds.bbci.co.uk/news/politics/rss.xml
15 http://feeds.bbci.co.uk/news/science_and_environment/rss.xml
16 http://feeds.bbci.co.uk/news/entertainment_and_arts/rss.xml
17 http://feeds.bbci.co.uk/news/world/africa/rss.xml
18 http://feeds.bbci.co.uk/news/world/asia/rss.xml
19 http://feeds.bbci.co.uk/news/world/eu

In [295]:
rss_entry = feed_data[36]
rss_entry

{'Link': 'http://www.bbc.co.uk/blogs/theeditors/rss.xml',
 'RssName': 'Editors Blog',
 'Source': 'bbc',
 'keymap': {'id': 'id',
  'link': 'link',
  'published_parsed': 'published_parsed',
  'summary': 'summary',
  'title': 'title'}}

In [296]:
contents = scrape_feeds.get_feed_contents(rss_entry)

In [269]:
from bs4 import BeautifulSoup as bs

In [284]:
for c in contents:
    flag = False
    if len(c['title']) > 200:
        flag = True
    if len(c['link']) > 200:
        flag = True
    if len(c['summary']) > 5000:
        flag = True
    if flag:
        print(c)

In [297]:
for i in range(len(contents)):
    print(contents[i]['summary'])
    print('\n')

As of Thursday, the Editors' blog will move to a different address on the BBC News website.
While this page will no longer be updated, it will stay here for reference. 


We've had a huge response to our class calculator this week, particularly across social media, following a major survey by BBC Lab UK. The survey suggests that traditional categories of working, middle and upper class are outdated and we all fit in to one of seven new classes. 

The class calculator - which lets you work out where you might fit in amongst the new categories - has attracted about six million page views on the BBC News site, making it the second most popular article of 2013 to date. (The most viewed article this year has been the helicopter crash in Vauxhall in January.) Nearly 1.9 million of those views have come from those of you accessing the site on mobiles and tablets.

But one thing that really stands out is how widely the story has been shared across social media, with more than 300,000 shares so

    3 of the 4 error types should be addressed now:
    + removed capture of story_id
    + Hard stop / truncate + exapnded field for title
    + Hard stop / truncate + expanded field for summary
        + also am now "preprocessing" html summary stuff w/ bs4 to go ahead and just extract the text from the html. This makes the entries shorter overall, thus reducing storage space.  however, things like links in the summaries are now removed...
    
    Expanded the "link" field in the db, but did not force truncate yet, which means stuff will still pop up...

In [288]:
bs('i ma a frong', 'html.parser').text

'i ma a frong'

In [298]:
mysql_utils.saveNewLinks(contents)

[64230,
 64231,
 64232,
 64233,
 64234,
 64235,
 64236,
 64237,
 64238,
 64239,
 64240,
 64241,
 64242,
 64243,
 64244]

### URL Errors

In [299]:
urle = errors[errors.err_class=="<class 'urllib.error.URLError'>"]
urle.shape

(3016, 5)

In [300]:
urle.tail()

,entry,day,entry_ind,is_error,err_class
2017-03-01 05:02:04,Feed https://phys.org/rss-feed/chemistry-news/...,2017-03-01,373,True,<class 'urllib.error.URLError'>
2017-03-01 05:02:05,Feed https://phys.org/rss-feed/chemistry-news/...,2017-03-01,373,True,<class 'urllib.error.URLError'>
2017-03-01 05:02:06,Feed https://phys.org/rss-feed/chemistry-news/...,2017-03-01,373,True,<class 'urllib.error.URLError'>
2017-03-01 05:02:07,Feed https://phys.org/rss-feed/chemistry-news/...,2017-03-01,373,True,<class 'urllib.error.URLError'>
2017-03-01 05:02:09,Feed https://phys.org/rss-feed/chemistry-news/...,2017-03-01,373,True,<class 'urllib.error.URLError'>


In [303]:
urle.entry.unique()[:10]

array([ "Feed https://phys.org/rss-feed/: <class 'urllib.error.URLError'>: (GeneralProxyError(),)",
       "Feed https://phys.org/rss-feed/editorials/: <class 'urllib.error.URLError'>: (GeneralProxyError(),)",
       "Feed https://phys.org/rss-feed/earth-news/: <class 'urllib.error.URLError'>: (GeneralProxyError(),)",
       "Feed https://phys.org/rss-feed/earth-news/earth-sciences/: <class 'urllib.error.URLError'>: (GeneralProxyError(),)",
       "Feed https://phys.org/rss-feed/earth-news/environment/: <class 'urllib.error.URLError'>: (GeneralProxyError(),)",
       "Feed https://phys.org/rss-feed/science-news/: <class 'urllib.error.URLError'>: (GeneralProxyError(),)",
       "Feed https://phys.org/rss-feed/science-news/archaeology-fossils/: <class 'urllib.error.URLError'>: (GeneralProxyError(),)",
       "Feed https://phys.org/rss-feed/science-news/economics-business/: <class 'urllib.error.URLError'>: (GeneralProxyError(),)",
       "Feed https://phys.org/rss-feed/science-news/mathem

In [304]:
roots = urle.entry.apply(lambda x: x[5:22])

In [308]:
len(roots.unique())

1

In [309]:
feed_data = utils.load_feedlist_data('physorg_feedlist.xml')

In [313]:
for i,f in enumerate(feed_data[:15]):
    print(str(i) + ' ' + f['Link'])

0 https://phys.org/rss-feed/
1 https://phys.org/rss-feed/editorials/
2 https://phys.org/rss-feed/earth-news/
3 https://phys.org/rss-feed/earth-news/earth-sciences/
4 https://phys.org/rss-feed/earth-news/environment/
5 https://phys.org/rss-feed/science-news/
6 https://phys.org/rss-feed/science-news/archaeology-fossils/
7 https://phys.org/rss-feed/science-news/economics-business/
8 https://phys.org/rss-feed/science-news/mathematics/
9 https://phys.org/rss-feed/science-news/sci-other/
10 https://phys.org/rss-feed/science-news/social-sciences/
11 https://phys.org/rss-feed/nanotech-news/
12 https://phys.org/rss-feed/nanotech-news/bio-medicine/
13 https://phys.org/rss-feed/nanotech-news/nano-materials/
14 https://phys.org/rss-feed/nanotech-news/nano-physics/


In [307]:
log[log.entry_ind==373].head()

,entry,day,entry_ind,is_error,err_class
2017-03-01 05:00:02,Retrieving contents...,2017-03-01,373,False,
2017-03-01 05:00:02,Total feeedss to visit: 250,2017-03-01,373,False,
2017-03-01 05:00:02,Creating 8 Tor openers...,2017-03-01,373,False,
2017-03-01 05:00:14,IP Used: b'65.19.167.132',2017-03-01,373,False,
2017-03-01 05:00:17,Total entries retrieved from https://www.thegu...,2017-03-01,373,False,


    No issues since beginning of March; this issue seems to be fixed now.

### HTTP Errors

In [344]:
htpe = errors['2017-03-02 00:00:00':].copy()
htpe = htpe[htpe.err_class == "<class 'urllib.error.HTTPError'>"]
htpe.shape

(182, 5)

In [345]:
htpe.tail()

,entry,day,entry_ind,is_error,err_class
2017-03-13 18:08:50,Feed http://www.ft.com/rss/management/connecte...,2017-03-13,489,True,<class 'urllib.error.HTTPError'>
2017-03-14 05:08:13,Feed http://www.ft.com/rss/world/mideast/econo...,2017-03-14,490,True,<class 'urllib.error.HTTPError'>
2017-03-14 05:09:12,Feed http://www.ft.com/rss/management/connecte...,2017-03-14,490,True,<class 'urllib.error.HTTPError'>
2017-03-14 06:08:19,Feed http://www.ft.com/rss/world/mideast/econo...,2017-03-14,491,True,<class 'urllib.error.HTTPError'>
2017-03-14 06:09:06,Feed http://www.ft.com/rss/management/connecte...,2017-03-14,491,True,<class 'urllib.error.HTTPError'>


In [346]:
roots = htpe.entry.apply(lambda x: x[5:22])

In [347]:
roots.unique()

array(['http://www.ft.com', 'http://feeds.foxn'], dtype=object)

In [348]:
len(htpe.entry.unique())

3

In [349]:
htpe.entry.unique()

array([ "Feed http://www.ft.com/rss/management/connected-business: <class 'urllib.error.HTTPError'>: ()",
       "Feed http://www.ft.com/rss/world/mideast/economy: <class 'urllib.error.HTTPError'>: ()",
       "Feed http://feeds.foxnews.com/foxnews/entertainment: <class 'urllib.error.HTTPError'>: ()"], dtype=object)

In [351]:
htpe[htpe.entry==list(htpe.entry.unique())[2]]

,entry,day,entry_ind,is_error,err_class
2017-03-13 13:00:44,Feed http://feeds.foxnews.com/foxnews/entertai...,2017-03-13,484,True,<class 'urllib.error.HTTPError'>


    The first two do not exist; they were removed.  The latter looks like an accident...
    There were others, but these stopped occurring after the url issue was fixed at the beginning of March 2017